# ANALYSIS ON TEACHING RATINGS DATA USING PYTHON

### I have used the teaching evaluation data from the University of Texas. The dataset comprises of 463 courses, in which we have information about the teaching evaluation score received by the instructor. We have information about the attributes of the instructor, as well as the characteristics of the course.

### In this project, I have answered various questions to gain insights on  how factors like minority, age, gender, beauty, nativity, tenure, etc., affects the evaluation of teaching.

## DATA DESCRIPTION

| Variable    | Description                                                                                                                                          |
| ----------- | ---------------------------------------------------------------------------------------------------------------------------------------------------- |
| minority    | Does the instructor belong to a minority (non-Caucasian) group?                                                                                      |
| age         | The professor's age                                                                                                                                  |
| gender      | Indicating whether the instructor was male or female.                                                                                                |
| credits     | Is the course a single-credit elective?                                                                                                              |
| beauty      | Rating of the instructor's physical appearance by a panel of six students averaged across the six panelists and standardized to have a mean of zero. |
| eval        | Course overall teaching evaluation score, on a scale of 1 (very unsatisfactory) to 5 (excellent).                                                    |
| division    | Is the course an upper or lower division course?                                                                                                     |
| native      | Is the instructor a native English speaker?                                                                                                          |
| tenure      | Is the instructor on a tenure track?                                                                                                                 |
| students    | Number of students that participated in the evaluation.                                                                                              |
| allstudents | Number of students enrolled in the course.                                                                                                           |
| prof        | Indicating instructor identifier.                                                                                                                    |


## IMPORT LIBRARIES


In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.api as sm

In [2]:
ratings_df=pd.read_csv('teaching_ratings.csv')

## DESCRIPTIVE STATISTICS

### Question 1: Calculate the percentage of visible minorities are tenure professors. Will you say that tenure status differed if teacher was a visible minority?


In [3]:
ratings_df1=ratings_df[ratings_df['tenure']=='yes']

tenure_count = ratings_df1.groupby('minority').agg({'tenure': 'count'}).reset_index()

tenure_count['percentage'] = 100 * tenure_count.tenure/tenure_count.tenure.sum()

print(tenure_count)

minority  tenure  percentage
0       no     307   85.041551
1      yes      54   14.958449


### Question 2: Does average age differ by tenure? Produce the means and standard deviations for both tenured and untenured professors.


In [4]:
result = ratings_df.groupby('tenure').agg({'age':['mean', 'std']}).reset_index()
print(result)

tenure        age           
               mean        std
0     no  50.186275   6.946372
1    yes  47.850416  10.420056


## PROBABILITY DISTRIBUTION

### Question 1: What is the probability of receiving an evaluation score greater than 3.3?


In [5]:
eval_mean = round(ratings_df['eval'].mean(), 3)
eval_sd = round(ratings_df['eval'].std(), 3)

prob_less_than = scipy.stats.norm.cdf((3.3 - eval_mean)/eval_sd)
print(1 - prob_less_than)

0.8957422041794154


### Question 2: What is the probability of receiving an evaluation score between 2 and 3?


In [6]:
x1 = 2
prob_less_than_2 = scipy.stats.norm.cdf((x1 - eval_mean)/eval_sd)
print(prob_less_than_2)

x2 = 3
prob_less_than_3 = scipy.stats.norm.cdf((x2 - eval_mean)/eval_sd)
print(prob_less_than_3)
##Percentage
round((prob_less_than_3 - prob_less_than_2)*100, 1)

0.00015910859015753364
0.03607280250262359
3.6


## HYPOTHESIS TESTING

### 1) T-TEST

### Question 1: Does tenure affect teaching evaluation scores?

*   Use α = 0.05


In [7]:
scipy.stats.ttest_ind(ratings_df[ratings_df['tenure'] == 'yes']['eval'],
                   ratings_df[ratings_df['tenure'] == 'no']['eval'], equal_var = True)
## The p-value is less than 0.05 that means that - we will reject the null hypothesis as there evidence that being tenured affects teaching evaluation scores

TtestResult(statistic=-2.8046798258451777, pvalue=0.005249471210198792, df=461.0)

### 2) CHI-SQUARE TEST

### Question 2: Is there an association between age and tenure?

*   Discretize the age into three groups 40 years and youngers, between 40 and 57 years, 57 years and older.
*   What is the conclusion at α = 0.01 and α = 0.05?


In [8]:
ratings_df.loc[(ratings_df['age'] <= 40), 'age_group'] = '40 years and younger'
ratings_df.loc[(ratings_df['age'] > 40)&(ratings_df['age'] < 57), 'age_group'] = 'between 40 and 57 years'
ratings_df.loc[(ratings_df['age'] >= 57), 'age_group'] = '57 years and older'

## Null Hypothesis: There is no association between age and tenure
## Alternative Hypothesis: There is an association between age and tenure

cont_table  = pd.crosstab(ratings_df['tenure'], ratings_df['age_group'])

scipy.stats.chi2_contingency(cont_table)
## At the α = 0.01, p-value is greater, we fail to reject null hypothesis as there is no evidence of an association between age and tenure
## At the α = 0.05, p-value is less, we reject null hypoothesis as there is evidence of an association between age and tenure

Chi2ContingencyResult(statistic=8.749576239010711, pvalue=0.012590809706820843, dof=2, expected_freq=array([[ 24.89416847,  26.87688985,  50.22894168],
       [ 88.10583153,  95.12311015, 177.77105832]]))

### 3) ANALYSIS ON VARIANCE (ANOVA)

### Question 3: Test for equality of variance for beauty scores between tenured and non-tenured instructors

*   Use α = 0.05


In [9]:
scipy.stats.levene(ratings_df[ratings_df['tenure'] == 'yes']['beauty'],
                   ratings_df[ratings_df['tenure'] == 'no']['beauty'], 
                   center='mean')
## Since the p-value is greater than 0.05, we will assume equality of variance of both groups

LeveneResult(statistic=0.4884241652750426, pvalue=0.4849835158609811)

## REGRESSION ANALYSIS

### Question 1: Does tenure affect beauty scores?
* Use α = 0.05


In [10]:
## Null Hypothesis: Mean beauty scores for tenured and non-tenured instructors are equal
## Alternative Hypothesis: There is a difference in mean beauty scores for tenured and non-tenured instructors

X = ratings_df['tenured_prof']
y = ratings_df['beauty']

X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Print out the statistics
model.summary()
## p-value is greater than 0.05, so we fail to reject the null hypothesis as there is no evidence that the mean difference of tenured and untenured instructors are different

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 beauty   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1689
Date:                Fri, 07 Mar 2025   Prob (F-statistic):              0.681
Time:                        19:22:12   Log-Likelihood:                -546.45
No. Observations:                 463   AIC:                             1097.
Df Residuals:                     461   BIC:                             1105.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0284      0.078      0.363      0.717      -0.125       0.182
tenured_prof    -0.0364      0.089     -0.411      0.681      -0.210       0.138
==============================================================================
Omnibus:                       23.184   Durbin-Watson:                   0.461
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.229
Skew:                           0.507   Prob(JB):                     9.03e-06
Kurtosis:                       2.583   Cond. No.                         4.05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Question 2: Does being an English speaker affect the number of students assigned to professors? 
* Use "allstudents"
* Use α = 0.05 and α = 0.1 


In [11]:
## Null Hypothesis: Mean number of students assigned to native English speakers vs non-native English speakers are equal
## Alternative Hypothesis: There is a difference in mean number of students assigned to native English speakers vs non-native English speakers

X = ratings_df['English_speaker']
y = ratings_df['allstudents']

X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

## At α = 0.05, p-value is greater, we fail to reject the null hypothesis as there is no evidence that being a native English speaker or a non-native English speaker affects the number of students assigned to an instructor.
## At α = 0.1, p-value is less, we reject the null hypothesis as there is evidence that there is a significant difference of mean number of students assigned to native English speakers vs non-native English speakers.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            allstudents   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.476
Date:                Fri, 07 Mar 2025   Prob (F-statistic):             0.0629
Time:                        19:23:42   Log-Likelihood:                -2654.2
No. Observations:                 463   AIC:                             5312.
Df Residuals:                     461   BIC:                             5321.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              29.6071     14.150      2.092      0.037       1.802      57.413
English_speaker    27.2158     14.598      1.864      0.063      -1.471      55.902
==============================================================================
Omnibus:                      429.792   Durbin-Watson:                   0.708
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10527.126
Skew:                           4.129   Prob(JB):                         0.00
Kurtosis:                      24.852   Cond. No.                         8.01
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Question 3: What is the correlation between the number of students who participated in the evaluation survey and evaluation scores?
* Use "students" variable


In [12]:
X = ratings_df['students']
y = ratings_df['eval']

X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

## R-square is 0.001, R will be √0.001, correlation coefficient is 0.03 (close to 0). There is a very weak correlation between the number of students who participated in the evaluation survey and evaluation scores

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   eval   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5806
Date:                Fri, 07 Mar 2025   Prob (F-statistic):              0.446
Time:                        19:25:07   Log-Likelihood:                -383.46
No. Observations:                 463   AIC:                             770.9
Df Residuals:                     461   BIC:                             779.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.9823      0.033    119.689      0.000       3.917       4.048
students       0.0004      0.001      0.762      0.446      -0.001       0.002
==============================================================================
Omnibus:                       15.259   Durbin-Watson:                   1.198
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.283
Skew:                          -0.456   Prob(JB):                     0.000291
Kurtosis:                       2.888   Cond. No.                         74.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""